In [1]:
import torch
from torch import optim

import math
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(0)

In [2]:
import torch
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/'+'/iris/iris.data', header=None)
df = df.sample(frac=1, random_state=0) # shuffle

df = df[df[4].isin(['Iris-virginica', 'Iris-versicolor'])]

# add label indices column
mapping = {k: v for v, k in enumerate(df[4].unique())}
df[5] = (2 * df[4].map(mapping)) - 1 # labels in {-1, 1}

# normalise data
alldata = torch.tensor(df.iloc[:, [0, 1, 2, 3]].values, dtype=torch.float)
alldata = (alldata - alldata.mean(dim=0)) / alldata.var(dim=0)

# create datasets
targets_tr = torch.tensor(df.iloc[:75, 5].values, dtype=torch.long)
targets_va = torch.tensor(df.iloc[75:, 5]. values, dtype=torch.long)
data_tr = alldata[:75]
data_va = alldata[75:]

In [3]:
def hinge_loss(y_pred, y_true):
    # YOUR CODE HERE
    yz = y_pred * y_true
    return torch.max(torch.zeros_like(yz), 1 - yz).sum()


def svm(x, w, b):
    h = (w*x).sum(1) + b
    return h

In [4]:
from torch.utils import data

In [5]:
NUM_EPOCHS = 100
BATCH_SIZE = 25

In [6]:
train = data.TensorDataset(data_tr, targets_tr)
val = data.TensorDataset(data_va, targets_va)
trainloader = data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
valloader = data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
def train_model(trainloader, valloader, w, b, optim_type=optim.SGD):
    optim = optim_type([w, b], lr=0.01, weight_decay=0.0001)

    train_loss = np.empty(NUM_EPOCHS*BATCH_SIZE)
    val_loss = np.empty(NUM_EPOCHS*BATCH_SIZE)
    i = 0
    j = 0
    for epoch in range(NUM_EPOCHS):
        for X, y_true in trainloader:
            optim.zero_grad()
            y_pred = svm(X, w, b)
            loss = hinge_loss(y_pred, y_true)
            loss.backward()
            optim.step()

            train_loss[i] = loss
            i += 1

        with torch.no_grad():
            for X, y_true in valloader:
                y_pred = svm(X, w, b)
                loss = hinge_loss(y_pred, y_true)
                val_loss[epoch]

                val_loss[j] = loss
                j += 1

    return w, b, train_loss, val_loss

def accuracy(X, y, w, b):
    correct = 0
    for i in range(len(y)):
        y_pred = svm(X[i], w, b)
        y_pred[y_pred > 0] = 1
        y_pred[y_pred < 0] = -1

        if y_pred == y[i]: correct += 1
    return float(correct)/len(y)


In [8]:
N = 100 # number of times experiment is repeated

In [9]:
val_acc_sgd = np.empty(N)
for i in range(N):
    w_sgd = torch.randn(1, data_tr.shape[1], requires_grad=True)
    b_sgd = torch.randn(1, requires_grad=True)

    w_sgd, b_sgd, train_loss_sgd, val_loss_sgd = train_model(trainloader, valloader, w_sgd, b_sgd, optim_type=optim.SGD)

    val_acc_sgd[i] = accuracy(data_va, targets_va, w_sgd, b_sgd)


In [10]:
val_acc_sgd

array([0.88, 0.88, 0.88, 0.88, 0.92, 0.92, 0.88, 0.92, 0.88, 0.88, 0.92,
       0.92, 0.88, 0.92, 0.88, 0.92, 0.92, 0.92, 0.88, 0.92, 0.88, 0.88,
       0.92, 0.88, 0.88, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.92, 0.88,
       0.92, 0.92, 0.92, 0.92, 0.92, 0.88, 0.88, 0.92, 0.92, 0.92, 0.88,
       0.88, 0.92, 0.92, 0.92, 0.88, 0.92, 0.88, 0.92, 0.88, 0.92, 0.88,
       0.88, 0.88, 0.92, 0.88, 0.92, 0.88, 0.92, 0.88, 0.88, 0.88, 0.92,
       0.92, 0.88, 0.88, 0.92, 0.88, 0.92, 0.88, 0.88, 0.92, 0.92, 0.92,
       0.88, 0.92, 0.88, 0.88, 0.92, 0.92, 0.92, 0.92, 0.92, 0.88, 0.92,
       0.88, 0.88, 0.88, 0.88, 0.88, 0.92, 0.92, 0.88, 0.88, 0.88, 0.88,
       0.92])

In [11]:
val_acc_sgd.mean()

0.9007999999999999

In [12]:
val_acc_adam = np.empty(N)
for i in range(N):
    w_adam = torch.randn(1, data_tr.shape[1], requires_grad=True)
    b_adam = torch.randn(1, requires_grad=True)

    w_adam, b_adam, train_loss_adam, val_loss_adam = train_model(trainloader, valloader, w_adam, b_adam, optim_type=optim.Adam)

    val_acc_adam[i] = accuracy(data_va, targets_va, w_adam, b_adam)

In [14]:
val_acc_adam.mean()

0.8912

In [26]:
df_accs = pd.DataFrame({"Mean": [val_acc_sgd.mean(), val_acc_adam.mean()], "Variance": [val_acc_sgd.var(), val_acc_adam.var()]}, index=['SGD', 'Adam'])

In [28]:
df_accs

,Mean,Variance
SGD,0.9008,0.000399
Adam,0.8912,0.000995


In [33]:
print(df_accs.to_latex(caption="Accuracy for 50 repeated experiemtns", label="tab:acc"))

\begin{table}
\centering
\caption{Accuracy for 50 repeated experiemtns}
\label{tab:acc}
\begin{tabular}{lrr}
\toprule
{} &    Mean &  Variance \\
\midrule
SGD  &  0.9008 &  0.000399 \\
Adam &  0.8912 &  0.000995 \\
\bottomrule
\end{tabular}
\end{table}

